In [7]:
import os 
from dotenv import load_dotenv
import requests
from google.oauth2 import service_account
import xmltodict
import json
import pandas as pd 
from requests import request
import pytz
from bs4 import BeautifulSoup
from typing import Dict
from google.oauth2 import service_account
from google.cloud import storage
fro

import pyspark
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
import os
import sys

from pyspark.sql.types import StructType, StructField, FloatType, BooleanType
from pyspark.sql.types import DoubleType, IntegerType, StringType





In [3]:
#load env variables
load_dotenv('./creds/.env', verbose=True, override=True)

True

In [3]:
print(os.environ.get("PYSPARK_DRIVER_PYTHON"))

None


In [4]:


'''
----------------
INIT VARIABLES
----------------
'''

#setting up entsoe variables
security_token = os.environ.get("SECURITY_TOKEN")
ENTSOE_URL = 'https://transparency.entsoe.eu/api'

#setting up GCP variables
service_account_file = os.environ.get("SERVICE_ACCOUNT_FILE")
credentials = service_account.Credentials.from_service_account_file(
    service_account_file
)
gcs_bucket = os.environ.get("CLOUD_STORAGE_BUCKET")
print(gcs_bucket)

entsoe_analytics_1009


In [ ]:



#setting up session
entsoe_session = requests.Session()

'''
----------------
SETTING UP FUNCTION CALLS 
----------------
'''

#format dates
def datetime_to_str(dtm: pd.Timestamp) -> str:
    #convert timezone to UTC if it's exist and it's not on UTC already
    if dtm.tzinfo is not None and dtm.tzinfo != pytz.UTC:
        dtm = dtm.tz_convert("UTC")
    fmt = '%Y%m%d%H%M'
    dtm_str = dtm.strftime(fmt)
    return dtm_str

#basic requests
def basic_requests(params: Dict, start:pd.Timestamp, end: pd.Timestamp) -> requests.Response:
    #setting up time intervals start and stop
    start_str = datetime_to_str(start)
    end_str = datetime_to_str(end)

    #setting up params and extending with custom parameter based
    base_params = {
        'securityToken': security_token,
        'periodStart': start_str,
        'periodEnd': end_str,
    }
    params.update(base_params)
    
    #seting up sesssion
    session = requests.Session()
    response = session.get(url=ENTSOE_URL, params=params)

    return response

# upload data to GCS
def upload_blob_to_gcs(bucket_name, contents, destination_blob_name):
    # Upload file to bucket"""

    # ID of GCS bucket
    # bucket_name =

    # the contents from memory to be uploaded to file
    # contents =

    # the ID of your GCS object
    # destination_blob_name =

    storage_client = storage.Client(credentials=credentials)
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    blob.upload_from_string(contents)


'''
----------------
EXTRACTION
----------------
'''
#for test, we'll be querying Actual Generation Output per Generation Unit

domain = '10YCZ-CEPS-----N'
params_requests = {
    'documentType': 'A73',
    'processType': 'A16',
    'in_Domain': {domain},
}
start=pd.Timestamp('202101011200', tz='Europe/Berlin')
end=pd.Timestamp('202101011300', tz='Europe/Berlin')

entsoe_data = basic_requests(params=params_requests, start=start, end=end)
# entsoe_dict = xmltodict.parse(entsoe_data.text)
# entsoe_json = json.dumps(entsoe_dict, indent=4)
# print(entsoe_json)
print(entsoe_data.text)

'''
----------------
LOAD
----------------
'''
# #upload to GCS
# landing_filename=f"entsoe_data_{start}.json"
# upload_blob_to_gcs(bucket_name=gcs_bucket, contents=entsoe_json, destination_blob_name=landing_filename)

#upload to GCS - XML
landing_filename=f"entsoe_data_{start}_{domain}.xml"
upload_blob_to_gcs(bucket_name=gcs_bucket, contents=entsoe_data.text, destination_blob_name=landing_filename)



In [5]:
print(os.environ["SPARK_HOME"])
SPARK_HOME = os.environ["SPARK_HOME"]

/opt/spark-3.3.1-bin-hadoop3-scala2.13


In [9]:

#setup sparksession for entry point
spark = SparkSession.builder.appName("gcp_playground").config("spark.jars", f"{SPARK_HOME}/jars/spark-xml_2.13-0.15.0.jar").getOrCreate()


22/12/23 09:53:58 WARN Utils: Your hostname, pop-os resolves to a loopback address: 127.0.1.1; using 192.168.18.2 instead (on interface wlp58s0)
22/12/23 09:53:58 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/12/23 09:53:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [19]:

# berangkat pak haji
# setup parameternya
gcs_bucket = gcs_bucket
# path = f"gs://{gcs_bucket}/{landing_filename}"
path ="/home/rafzul/projects/entsoe-pipelines/sample.xml"

print(path)
#spark read json
df = spark.read.format('xml').options(rowTag='book', rootTag='books', inferSchema=True).load('/home/rafzul/projects/entsoe-pipelines/sample.xml')
df.show()

/home/rafzul/projects/entsoe-pipelines/sample.xml
+-----+--------------------+--------------------+---------------+-----+------------+--------------------+
|  _id|              author|         description|          genre|price|publish_date|               title|
+-----+--------------------+--------------------+---------------+-----+------------+--------------------+
|bk101|Gambardella, Matthew|\n\n\n         An...|       Computer|44.95|  2000-10-01|XML Developer's G...|
|bk102|          Ralls, Kim|A former architec...|        Fantasy| 5.95|  2000-12-16|       Midnight Rain|
|bk103|         Corets, Eva|After the collaps...|        Fantasy| 5.95|  2000-11-17|     Maeve Ascendant|
|bk104|         Corets, Eva|In post-apocalyps...|        Fantasy| 5.95|  2001-03-10|     Oberon's Legacy|
|bk105|         Corets, Eva|The two daughters...|        Fantasy| 5.95|  2001-09-10|  The Sundered Grail|
|bk106|    Randall, Cynthia|When Carla meets ...|        Romance| 4.95|  2000-09-02|         Lover Bir

In [6]:
with open("entsoe_analytics_1009/entsoe_data_2021-01-01 12:00:00+01:00_10YCZ-CEPS-----N.xml") as source_file:
    xml_file_dict = [xmltodict.parse(source_file.read())]
    xml_json_dump = json.dumps(xml_file_dict, indent=4)
    
with open("entsoe_json.json", "w") as outfile:
    outfile.write(xml_json_dump)

In [24]:
#inferring schema and get the data type of each column and turn it into spark dataframe
datatype_infer = pd.DataFrame(xml_file_dict[0])
datatype_infer

,GL_MarketDocument
@xmlns,urn:iec62325.351:tc57wg16:451-6:generationload...
TimeSeries,"[{'mRID': '1', 'businessType': 'A01', 'objectA..."
createdDateTime,2022-12-20T23:55:03Z
mRID,087f7c50ecdb477394b0045fdc1f1173
process.processType,A16
receiver_MarketParticipant.mRID,"{'@codingScheme': 'A01', '#text': '10X1001A100..."
receiver_MarketParticipant.marketRole.type,A33
revisionNumber,1
sender_MarketParticipant.mRID,"{'@codingScheme': 'A01', '#text': '10X1001A100..."
sender_MarketParticipant.marketRole.type,A32


In [39]:
with open("entsoe_analytics_1009/entsoe_data_2021-01-01 12:00:00+01:00_10YCZ-CEPS-----N.xml") as source_file:
    xml_file_dict = [xmltodict.parse(source_file.read())]["GL_MarketDocument"]
    df = spark.createDataFrame([xml_file_dict])
    df.show()

<>:2: SyntaxWarning: list indices must be integers or slices, not str; perhaps you missed a comma?
<>:2: SyntaxWarning: list indices must be integers or slices, not str; perhaps you missed a comma?
/tmp/ipykernel_6069/2241073966.py:2: SyntaxWarning: list indices must be integers or slices, not str; perhaps you missed a comma?
  xml_file_dict = [xmltodict.parse(source_file.read())]["GL_MarketDocument"]
/tmp/ipykernel_6069/2241073966.py:2: SyntaxWarning: list indices must be integers or slices, not str; perhaps you missed a comma?
  xml_file_dict = [xmltodict.parse(source_file.read())]["GL_MarketDocument"]
/tmp/ipykernel_6069/2241073966.py:2: SyntaxWarning: list indices must be integers or slices, not str; perhaps you missed a comma?
  xml_file_dict = [xmltodict.parse(source_file.read())]["GL_MarketDocument"]


TypeError: list indices must be integers or slices, not str